In [179]:
import numpy as np
from   statistics              import mode
from   sklearn.metrics         import *
from   sklearn.base            import ClassifierMixin
from   sklearn.datasets        import make_classification
from   sklearn.model_selection import train_test_split

from sklearn.tree          import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.svm           import SVC

In [180]:
data = make_classification(n_samples=10_000,n_classes=2,n_features=10)
X = data[0]
y = data[1]
xtrain ,xtest ,ytrain ,ytest = train_test_split(X,y,random_state=42,test_size=0.2)

In [191]:
xtrain.shape

(8000, 10)

### Metodo do voting

In [181]:
class VotingCL(ClassifierMixin):
    models   = []
    predicts = []
    result   = []
    def __init__(self,Classifier:list[list]=None,type:['soft','mode']='mode'):
        self.Classifier = Classifier
        self.type       = type
        self.length     = len(Classifier)

    def fit(self,X,y):
        for modeloCl in self.Classifier:     
            self.models.append(modeloCl[0].fit(X,y))
    
    def predict(self,X):
        for xtest in X:
            pred = []
            for modeloCl in self.models:
                if (self.type == 'soft'):
                    pred.append(modeloCl.predict_proba([xtest])[0])
                else:
                    pred.append(modeloCl.predict([xtest])[0])
            self.predicts.append(pred)
            
        for pred in self.predicts:
            if self.type == 'soft':
                sum1 ,sum2 = 0,0
                for zero, one in pred:
                    sum1 += zero
                    sum2 += one
                
                self.result.append([sum1/self.length,sum2/self.length])
            else:
                self.result.append(mode(pred))    
                
        if self.type == 'soft':
            return np.array([ i.index(max(i)) for i in self.result])
        return np.array(self.result)

In [182]:
voting = VotingCL(
    Classifier = [
        [LogisticRegression()],
        [DecisionTreeClassifier()],
        [SVC(probability=True)]
    ],
    type='soft'
)
voting.fit(xtrain,ytrain)

In [183]:
pred = (voting.predict(xtest))

In [184]:
pred

array([0, 0, 0, ..., 0, 0, 1])

In [185]:
pred.shape

(2000,)

In [186]:
accuracy_score(ytest,pred)

0.9125

### Bagging e Pasting

In [188]:
from sklearn.ensemble import BaggingClassifier

In [235]:
bagging = BaggingClassifier(DecisionTreeClassifier(),n_estimators=100,max_samples=100,max_features=1,bootstrap=True)

In [236]:
bagging.fit(xtrain,ytrain)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_features=1,
                  max_samples=100, n_estimators=100)

In [238]:
bagging.max_features

1

In [237]:
accuracy_score(ytest,bagging.predict(xtest))

0.813

### Floresta aleatoria

In [239]:
from sklearn.ensemble import RandomForestClassifier

In [241]:
random = RandomForestClassifier(100)
random.fit(xtrain,ytrain)

RandomForestClassifier()

In [243]:
accuracy_score(ytest,random.predict(xtest))

0.928

In [244]:
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor

In [255]:
ada = AdaBoostClassifier(LogisticRegression(),n_estimators=10,algorithm='SAMME.R',learning_rate=0.2)

In [256]:
ada.fit(xtrain,ytrain)
accuracy_score(ytest,ada.predict(xtest))

0.903